![LogoTremplin](images/tremplinColorSmall.png)

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.

![Licence](images/Licence.jpg)

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan

------------------------------------------------------------

# La detection de présence : Projet EyeKAM

Projet conduit par trois élèves au Lycée Camille Saint Saëns, encadrés par Marie Pierre Cribier, Pascal Delacour, Eric Le Jan. 

[la page du capteur](https://learn.adafruit.com/pir-passive-infrared-proximity-motion-sensor?view=all)

le capteur recto et verso

|||
|:--:|:--:|
|![recto](images/capteurPIRmotionFace.png)|![verso](images/capteurPIRmotion.png)| 

Sitographie : 

[la page utilisée](https://learn.adafruit.com/adafruits-raspberry-pi-lesson-12-sensing-movement) pour mettre en place ce petit programme

[La page utilisée](https://openclassrooms.com/courses/apprenez-a-programmer-en-python/le-temps) pour le module time

[La page utilisée](https://docs.python.org/3/library/shutil.html#shutil.copy) pour copier le fichier de capture dans le dossier accessible sur le web.

[La page utilisée](https://www.developpez.net/forums/d1341218/autres-langages/python-zope/general-python/interrompre-boucle-while-l-appuie-d-touche/) pour interrompre proprement le kernel.

Pour les leds l'ipynb utilisé est dans la base CahierDeProgrammes puis dans JeuxDeLumiere "une led"

Les élèves disposent d'une Raspbery Pi3 d'une bredboard du capteur de mouvement, un capteur de température et un capteur de lumière. 

## Une première approche : Détecter la présence en allumant des leds et afficher une information

L'objectif poursuivi est faire de comprendre qu'il faut une interraction entre la Raspi et l'environnement. 
Le projet doit contenir du code python, on ne peut donc pas se contenter de récupérer le flux streamé de la raspi.

Le premier programme permet de tester l'allumage et l'extinction des leds rouge et verte.

Le second programme tourne en boucle l'arrêt se produit à l'interruption du kernel. Le capteur est alimenté en 5 V via le T cobbler sur le bredboard.

## Test des leds

Une led verte et une led rouge sont mises en oeuvre.
Les numéro des pins correspondent à ceux indiqués sur le T Cobbler GPIO Extension Board.

In [ ]:
#test de l'alumage des leds rouges et vertes
#On importe la bibliothèque de base de temps, la bibliothèque mraa (compatibilité avec RASPI, Arduino, Intel)
import time
import mraa
# On defini le pin 11 pour la led (Rouge)
led = mraa.Gpio(11)
time.sleep(0.2)
#On définit le pin 32 pour la ledverte
ledVerte = mraa.Gpio(32)
time.sleep(0.2)
#on définit les pin en sortie
status = led.dir(mraa.DIR_OUT)
time.sleep(0.3)
statusVert = ledVerte.dir(mraa.DIR_OUT)
time.sleep(0.3)
#On crée une fonction pour le test (clignotement alterné de 0,5 secondes)
def testclignotant():
    for i in range(5):
        status = led.write(1)
        time.sleep(0.5)
        status = led.write(0)
        time.sleep(0.5)
        statusVert = ledVerte.write(1)
        time.sleep(0.5)
        statusVert = ledVerte.write(0)

## Lancement de la fonction clignottement.

In [ ]:
testclignotant()

## Une solution pour arrêter le programme 

Le bouton d'interruption du kernel stoppe le compte. 

In [ ]:
import time
import os
def arret():
    print(" pour arrêter cliquez sur le bouton d'interruption") 
    x = 0
    while 1:
        try:
            x += 1
            time.sleep(2)
        except KeyboardInterrupt:
            print ('Arrêté à :', time.strftime("%A %d %B %Y %H:%M:%S"), "avec x = ",x)
            break

In [ ]:
#pour arrêter le décompte cliquez sur le bouton d'interruption

arret()

## Test du fonctionnement du détecteur de mouvement

On place le détecteur sur le pin 18.
Ce pin est défini en entrée. 

On va utiliser la fonction read pour informer de la présence ou non d'une personne dans la pièce.

On teste en important un nombre et une fréquence des mesures.

On arrête le programme avec le bouton d'interruption.


In [ ]:
#On importe la bibliothèque de base de temps, la bibliothèque mraa (compatibilité avec RASPI, Arduino, Intel)
import time
import mraa
# On defini le pin 11 pour la led Rouge
led = mraa.Gpio(11)
time.sleep(0.2)
#On définit le pin 32 pour la led Verte
ledVerte = mraa.Gpio(32)
time.sleep(0.2)
#on définit les pin en sortie
status = led.dir(mraa.DIR_OUT)
time.sleep(0.3)
statusVert = ledVerte.dir(mraa.DIR_OUT)
time.sleep(0.3)
#on place le detecteur sur le pin 18  
pir = mraa.Gpio(18)
#on place le pin 18 en mode IN
statuspir = pir.dir(mraa.DIR_IN)

# on attend un signal détecté et on affiche une info et une couleur de led à la façon des entrées/sorties de parking à une seule voie. 
      
etatpir = 0
print("Appuyez sur le bouton d'interruption pour arrêter la détection avant la fin des mesures")
while 1:
    try:
        NbreDeMesures = int(input("entrez le nombre de mesures souhaitées :"))
        FrequenceDesMesures = int(input("entrez l'intervalle entre 2 mesures en secondes"))
        for i in range(NbreDeMesures):
            etatpir = pir.read()
            if etatpir == -1:
                etatpir = 0
            elif etatpir == 1:
                print("il y a quelqu'un ! …",pir.read(),"…",end='')
                statusVert = ledVerte.write(0)
                status = led.write(1)
                time.sleep(FrequenceDesMesures)    
            else:
                print("il n'y a persone ! …",pir.read(),"…",end='')
                status = led.write(0)
                statusVert = ledVerte.write(1)
                time.sleep(FrequenceDesMesures)

    except KeyboardInterrupt:
        print("")
        print("")
        print ('Arrêté à :', time.strftime("%A %d %B %Y %H:%M:%S"), "avec etat du détecteur = ",pir.read())
        status = led.write(0)
        statusVert = ledVerte.write(0)
        break    

## Capture d'une photo avec la picam

Il s'agit ici de capturer une image jpeg, de la déplacer dans le dossier prévu sur la RASPI qui permet de la visualiser en pointant dessus depuis le navigateur. 

Quelques étapes préalables : 

- créer la hiérarchie de dossier sites/media/images/ dans le dossier /var/www/html/

- Donner tous les droits à tout le monde sur le dossier media (dans un premier temps) 

- régler le timelaps pour les captures (exprimé en secondes) 

- régler le nombre de captures souhaitées (entier compris entre 1 et 20) (pas de vérification pour l'instant)


In [ ]:
from time import *
from picamera import PiCamera
import shutil
import os
FrequenceCapture = int(input("saisissez la fréquence des captures en secondes entre 1 et 20 secondes : " ))
while FrequenceCapture >20 or FrequenceCapture <1 :
    FrequenceCapture = int(input("saisissez la fréquence des captures en secondes entre 1 et 20 secondes : " ))
NombreDeCaptures = int(input("saisissez le nombre de captures souhaitées entre 1 et 4: "))
while NombreDeCaptures >4 or NombreDeCaptures <1 :
    NombreDeCaptures = int(input("saisissez le nombre de captures entre 1 et 4 : " ))
camera = PiCamera()
camera.resolution = (400, 300)
camera.start_preview()
# Camera warm-up time
sleep(2)
u=0
for i in range(NombreDeCaptures):
    u = str(i)+'.jpg'
    print(u)
    camera.capture(u)
    shutil.copy(u,'/var/www/html/sites/media/images/', follow_symlinks=True)
    sleep(FrequenceCapture)
camera.stop_preview()
camera.close()

# Affichage des images capturées : essai en markdown 

Il faut accéder au répertoire de la raspi qui contient les images. Une façon simple est d'utiliser nginx et donc de pointer sur un répertoire en lecture pour tous situé dans l'espace var/www/

Il faut naturellement utiliser l'IP de la raspi sur le réseau sur lequel on est aussi, ou si on est à l'extérieur utiliser une connexion VPN pour accéder au réseau de la RASPI.

L'exemple ci-dessous utilise la localisation dans l'espace de publication de ce cahier.

|images|images|
|:---:|:---:|
|![image test](images/tremplinColorSmall.png)|![image test](images/tremplinColorSmall.png) |
|![image test](images/tremplinColorSmall.png)|![image test](images/tremplinColorSmall.png) |

# Affichage en utilisant un champ python et l'interprétation du html :

on peut appeler une fonction de python qui interprète le langage html.

ici encore le code doit être adapté pour que l'adresse de l'image soit celle qui correspond au lieu de l'enregistrement sur la raspi.

In [2]:
%%html
<p>insertion image</p>
<center>
<table align="center" border="2" cellpadding="2" cellspacing="2" style="height:100px;width:500px;">
	<caption>Captures</caption>
	<tbody>
		<tr>
			<td><img alt="" src="images/tremplinColorSmall.png" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
			<td><img alt="" src="images/tremplinColorSmall.png" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
		</tr>
		<tr>
			<td><img alt="" src="images/tremplinColorSmall.png" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
			<td><img alt="" src="images/tremplinColorSmall.png" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
		</tr>
	</tbody>
</table>
</center>
<center>&nbsp;</center>

<center>&nbsp;</center>

<p>&nbsp;</p>

,
,


# On passe maintenant à la capture avec détection et la notification par mail.

Il faut enclencher la capture quand le détecteur est à 1 

On doit recapturer si le capteur passe de nouveau à 1 

Un compteur permet de modifier les noms de fichier de captures

On définit la fonction capture.

On utilise la fonction SendMail() pour envoyer la capture sur une adresse email.

On utilise l'appui sur le bouton interrupt de jupyterhub pour arrêter le programme.

On affiche plusieurs infos. 

In [ ]:
from picamera import PiCamera
import time
import shutil
import os
import mraa
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

# On defini le pin 11 pour la led Rouge
led = mraa.Gpio(11)
time.sleep(0.2)
#On définit le pin 32 pour la led Verte
ledVerte = mraa.Gpio(32)
time.sleep(0.2)
#on définit les pin en sortie
status = led.dir(mraa.DIR_OUT)
time.sleep(0.3)
statusVert = ledVerte.dir(mraa.DIR_OUT)
time.sleep(0.3)
#on place le detecteur sur le pin 18  
pir = mraa.Gpio(18)
#on place le pin 18 en mode IN
statuspir = pir.dir(mraa.DIR_IN)

compteur = 0
u = 0
etatpir = 0
chemin = 'chemin'

def capture():
    camera = PiCamera()
    camera.resolution = (400, 300)
    camera.start_preview()
    # Camera warm-up time
    time.sleep(1)
    camera.capture(u)
    time.sleep(0.5)
    shutil.copy(u,'/var/www/html/sites/media/images/', follow_symlinks=True)
    camera.stop_preview()
    camera.close()

def SendMail():
    chemin = "/var/www/html/sites/media/images/"+u
    print(chemin)
    img_data = open(chemin, 'rb').read()
    msg = MIMEMultipart()
    msg['Subject'] = 'Alerte intrusion'
    msg['From'] = 'exemple@gmail.com'
    msg['To'] = 'exemple@gmail.com , exemple2@mail.fr'
    text = MIMEText("légende de l'image")
    msg.attach(text)
    
    image = MIMEImage(img_data, name=os.path.basename(chemin))
    msg.attach(image)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login("exemple@gmail.com", "motdepassedeconnexion")
    s.sendmail("from", "to", msg.as_string())
    s.quit()


while 1:
    try:
        etatpir = pir.read()
        if etatpir == -1:
            etatpir = 0
        elif etatpir == 1:
            statusVert = ledVerte.write(0)
            status = led.write(1)
            u = str(compteur)+'detect.jpg'
            capture()
            print("il y a quelqu'un ! …",pir.read(),"…",end='')
            SendMail()
            compteur = compteur + 1
            time.sleep(1)    
        else:
            print("il n'y a persone ! …",pir.read(),"…",end='')
            status = led.write(0)
            statusVert = ledVerte.write(1)
            time.sleep(1)

    except KeyboardInterrupt:
        print("")
        print ('Arrêté à :', time.strftime("%A %d %B %Y %H:%M:%S"), "avec état du détecteur = ",pir.read())
        status = led.write(0)
        statusVert = ledVerte.write(0)
        break    


print("")
print("état du détecteur = ",etatpir)
print("nombre de détections = ", compteur)

In [ ]:
print(compteur)

## On affiche la dernière image capturée

Pour la repérer on utilise le dernier état du compteur.

|Image de l'intrus|
|:---:|
|![intrus](images/tremplinColorSmall.png)|